In [1]:
import numpy as np
import dash_script


First, import data. This data was obtained using the [astroML package](https://www.astroml.org/).

In [2]:
data = np.load('spec4000.npz')
wavelength = np.load('wavelength.npz')['arr_0']
list(data.keys())

['norms',
 'spectra',
 'log_OIII_Hb',
 'zerr',
 'mu',
 'coeffs',
 'z',
 'coeff1',
 'lineindex_cln',
 'spec_cln',
 'coeff0',
 'evals',
 'mask',
 'log_NII_Ha',
 'evecs']

Now extract spectra and x and y coordinates. In this example, we are going to plot a [BPT diagram](https://ned.ipac.caltech.edu/level5/Glossary/Essay_bpt.html). 
<br>The purpose of this module is to plot galaxies in an x-y coordinate (such as BPT), and then hover over a point to see its spectrum in real time.

In [3]:
spectra = data['spectra']
n = spectra.shape[0]

n2_ha = data['log_NII_Ha']
o3_hb = data['log_OIII_Hb']

To illustrate more ways of using the dash plotting function, the code below creates additional plots that zoom-in on certain emission lines.

In [26]:
lines_to_mask = ['OII_doublet', 'OIII (5007)', 'H_alpha']
lines_waves = [3728.5, 5008.2, 6564.6]
masking_windows = [10, 7, 10]
masked_spectra = np.zeros_like(spectra)
for i in range(n):
    masked_spectra[i,:] = spectra[i,:]
    for l in range(len(lines_to_mask)):
        # np.searchsorted is used to find the indices of the masking window edges
        i1, i2 = np.searchsorted(wavelength, (lines_waves[l]-masking_windows[l], lines_waves[l]+masking_windows[l]))
        x0, x1 = wavelength[i1-1], wavelength[i2+1]
        y0, y1 = spectra[i,i1-1], spectra[i,i2+1]
        # the emission line region is replaces by linearly interpolating the underlying continuum
        masked_spectra[i,i1:i2] = np.interp(wavelength[i1:i2], [x0,x1], [y0,y1])
        # the below if statement is to prevent masking absoprtion lines
        if np.average(masked_spectra[i,i1:i2]) > np.average(spectra[i,i1:i2]):
                masked_spectra[i,i1:i2] = spectra[i,i1:i2]

# This dictionary is used to tell the dash function which wavelengths to zoom in on and show in subplots titled by the keys.
# The values must be a list/array of size len(x), so if you want to use the same value for each point, you can simply do [value]*number of points
zoom = {}
for l in range(len(lines_to_mask)):
     zoom.update({lines_to_mask[l]: [lines_waves[l]]*n})

The below function call demonstrates the features available. <br>
The limits on the x and y axes can be set. <br>
A custom color coding can be used in the x,y plane, with any colormap that is allowed by Plotly.<br>
Multiple spectra can be inputted as a list, and each spectrum's color and name (to be shown in the legend) can be given.<br>
Zoom-in subplots can be added that zoom in on particular parts of the spectra.<br>

In [27]:
x1 = np.arange(-2,0,0.1)
y1 = 0.61/(x1-0.05) + 1.3
x2 = np.arange(-2,0.4,0.1)
y2 = 0.61/(x2-0.47) + 1.19

first_line = {
    'x': x1,
    'y': y1,
    'color': 'magenta',
    'name': 'K03',
    'line_dash': 'solid',
    'line_width': 8
}
second_line = {
    'x': x2,
    'y': y2,
    'color': 'magenta',
    'name': 'K01',
    'line_dash': 'dash',
    'line_width': 8
}

In [28]:
app = dash_script.dash_plot_spectra(
    x={'n2_ha': n2_ha},
    y={'o3_hb': o3_hb},
    xlim=[-1.5,1],
    ylim=[-1.2,1],
    color_code={'redshift': np.clip(data['z'],a_min=0,a_max=0.2)},
    cmap='Viridis',
    additional_lines=[first_line, second_line],
    spectra=[spectra, masked_spectra],
    wavelength=[wavelength, wavelength],
    spec_colors=['rgba(0,256,100,0.5)', 'rgba(256,165,0,1)'],
    spec_names=['Observed', 'masked'],\
    zoom=zoom,
    zoom_windows=[15,15,15],
    zoom_extras = [{'OII_doublet flux': spectra[:,np.searchsorted(wavelength, lines_waves[0])]},
                    {'OIII(5007) flux': spectra[:,np.searchsorted(wavelength, lines_waves[1])-1]},
                    {'H_alpha flux': spectra[:,np.searchsorted(wavelength, lines_waves[2])-1]}]
)


In [29]:
if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/
